In [45]:
!pip install datasets

In [46]:
!pip install transformers datasets

In [47]:
from datasets import load_dataset
import torch
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from scipy.optimize import brentq
from scipy.interpolate import interp1d
from transformers import AutoModelForSequenceClassification, Wav2Vec2Processor
import soundfile as sf

In [48]:
import os

# Define the path to the dataset directory in your Google Drive
dataset_dir = '/content/drive/MyDrive/test'

# Function to recursively collect all WAV files in the directory
def collect_wav_files(directory):
    wav_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.wav'):
                wav_files.append(os.path.join(root, file))
    return wav_files

# Collect all WAV files in the dataset directory
all_wav_files = collect_wav_files(dataset_dir)

# Display the number of WAV files found
print("Number of WAV files found:", len(all_wav_files))

# Display example file paths
print("Example file paths:")
for i in range(5):  # Display paths of the first 5 files
    print(all_wav_files[i])


Number of WAV files found: 505
Example file paths:
/content/drive/MyDrive/test/id10309/_z_BR0ERa9g/00005.wav
/content/drive/MyDrive/test/id10309/Zx-zA-D_DvI/00003.wav
/content/drive/MyDrive/test/id10309/vobW27_-JyQ/00002.wav
/content/drive/MyDrive/test/id10309/vobW27_-JyQ/00015.wav
/content/drive/MyDrive/test/id10309/V_tzoDBiP8A/00003.wav


In [49]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

In [51]:
!pip install fairseq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 31.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [53]:
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

# Load the pre-trained model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# Save the processor and model together in a checkpoint file
checkpoint = {
    'processor': processor,
    'model': model.state_dict()  # Save only the model state dictionary
}
torch.save(checkpoint, "/content/drive/MyDrive/wav2vec_checkpoint.pt")


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

In [64]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

In [65]:
# Load the pre-trained model and processor for wav2vec2-large
processor_large = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
model_large = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")

# Save the processor and model together in a checkpoint file
checkpoint_large = {
    'processor': processor_large,
    'model': model_large.state_dict()  # Save only the model state dictionary
}
torch.save(checkpoint_large, "/content/drive/MyDrive/wav2vec2_large_checkpoint.pt")

# Load the pre-trained model and tokenizer for DistilBERT
tokenizer_distilbert = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model_distilbert = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

# Save the tokenizer and model together in a checkpoint file
checkpoint_distilbert = {
    'tokenizer': tokenizer_distilbert,
    'model': model_distilbert.state_dict()
}
torch.save(checkpoint_distilbert, "/content/drive/MyDrive/distilbert_checkpoint.pt")

Some weights of the model checkpoint at facebook/wav2vec2-large-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You s

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [74]:
!pip install datasets
!pip install transformers datasets

In [75]:
import torch
import numpy as np
import os
from sklearn.metrics import roc_curve
from scipy.optimize import brentq
from scipy.interpolate import interp1d
from datasets import load_dataset
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, DistilBertTokenizer, DistilBertForSequenceClassification
import soundfile as sf
import librosa

In [76]:
# Function to recursively collect all WAV files in the directory
def collect_wav_files(directory):
    wav_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.wav'):
                wav_files.append(os.path.join(root, file))
    return wav_files

In [102]:
import librosa
import numpy as np
def extract_features(audio):
    # Compute MFCCs
    mfcc = librosa.feature.mfcc(y=audio, sr=None, n_mfcc=13)

    return mfcc



In [103]:
def compute_similarity_scores(audio_files, model):
    scores = []
    labels = []
    for i in range(len(audio_files)):
        for j in range(i + 1, len(audio_files)):
            # Load audio files
            audio_i, sr_i = librosa.load(audio_files[i], sr=None)
            audio_j, sr_j = librosa.load(audio_files[j], sr=None)

            # Extract features from audio files
            features_i = extract_features(audio_i, sr_i)
            features_j = extract_features(audio_j, sr_j)

            # Compute similarity score (example: dot product of features)
            similarity_score = np.dot(features_i.flatten(), features_j.flatten())
            scores.append(similarity_score)
            labels.append(1 if i == j else 0)  # 1 if same speaker, 0 if different speakers
    return scores, labels


In [104]:
# Function to calculate EER
def calculate_eer(scores, labels):
    fpr, tpr, thresholds = roc_curve(labels, scores, pos_label=1)
    eer = brentq(lambda x: 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    return eer * 100  # Convert EER to percentage

In [105]:
def compute_eer_for_model(model_name):
    # Load the pre-trained model and processor
    if model_name == "wav2vec2-base":
        processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
        model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
    elif model_name == "wav2vec2-large":
        processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
        model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")
    elif model_name == "distilbert-base":
        tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
        model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
    # Add other models as needed

    # Save the processor and model together in a checkpoint file
    if model_name != "distilbert-base":
        checkpoint = {
            'processor': processor,
            'model': model.state_dict()
        }
        torch.save(checkpoint, f"/content/drive/MyDrive/{model_name}_checkpoint.pt")

    # Define the path to the dataset directory
    dataset_dir = '/content/drive/MyDrive/test'

    # Collect all WAV files in the dataset directory
    all_wav_files = collect_wav_files(dataset_dir)

    # Compute similarity scores and labels for all pairs of audio files
    scores, labels = compute_similarity_scores(all_wav_files, model)

    # Calculate EER
    eer = calculate_eer(scores, labels)
    print(f"Equal Error Rate (EER) for {model_name}: {eer:.2f}%")

In [ ]:
print("EER for wav2vec2-base:", eer_wav2vec2_base

In [ ]:
print("EER for wav2vec2-large:", eer_wav2vec2_large)

In [ ]:
print("EER for distilbert-base:", eer_distilbert_base)

In [ ]:
audio_files = load_audio_files(dataset_dir)

In [ ]:
# Set maximum length for MFCC feature vectors
max_length = 1000

In [ ]:
# Compute similarity scores for all pairs of audio files
scores, labels = compute_similarity_scores(audio_files, max_length)

In [ ]:
eer = calculate_eer(scores, labels)
print("Equal Error Rate (EER): {:.2f}%".format(eer))